# Project - 1 : Apache Spark 

Instructor: Ramesh Yerraballi
TA: Madhumitha Sakthi
Semester: Fall 2019
Due Date: 11:59pm, Monday 9/16

This project is based on Map-Reduce Framework. In these you will get to work with Spark and will get to know how 
does spark work, what functionalities does spark provide, what does map-reduce framework do and why is it useful. 

In this project you will be implementing a basic song recommender system. You will be given a dataset where there are multiple csv files. These csv files have data corresponding to song play count and song information.

The data you would be using will be provided in a zip file along with this notebook. The __msd.zip__ archive contains '_kaggle_visible_evaluation_triplets.txt_'. We will be using the visible part of the testing data to understand the working on Apache Spark.  The user's listening history is provided as: (user, song, play count). In another file, '_kaggle_songs.txt_', each song is marked using an index for easier representation of songs. 



What to turn in:
A zip folder which will have:
- Jupyter Notebook
- A brief report on what features you used for recommendation. And a brief explanation of flow of your code. For example,  what RDD does what or, why it was created.
- datasets folder with the csv files you are using in your notebook.
- Notebook should use relative path to the csv files in datasets folder.
- Name of the zip folder - <your\_name>\_<your\_partner_name>.zip


This project consists of 4 questions. 
1. Create an RDD with _msd_evalutation_triplets.txt_ and replace the song name with the song index from _msd_songs.txt_. Identify the number of songs that do not have any rating. 
2. Generate song ratings based on the song play count as a normalized score between 0 and 1. 
3. Identify the popular song based on this rating and recommend songs to user, given user id based on the algorithm used in Movie recommender system from class. 
4. Using Cosine similarity function, identify pair-wise similarity between each pair of users and generate the top 5 most similar users without an overlap in users. 

The above list is the higer level idea about the questions. 

In [1]:
### Starter code ####

##### These lines are to tell jupyter where to find Apache Spark ####
import findspark
findspark.init('C:\spark')
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local[*]").setAppName("Songs")
# There are two configurable parameters
# 1. A cluster URL, namely  local  in this example, which tells Spark how to connect
# to a cluster.  local  is a special value that runs Spark on one thread on the local
# machine, without connecting to a cluster.
# 2. An application name, namely  Movies  in this example. This will identify your
# application on the cluster manager’s UI if you connect to a cluster.
sc = SparkContext(conf = conf)
##### These lines are to tell jupyter where to find Apache Spark ####

In [2]:
## Read triplet file into RDD
triplet_rdd = sc.textFile(r"msd\kaggle_visible_evaluation_triplets.txt") \
    .map(lambda line: line.split("\t")) 

## Step 1: 
Replace song name with song index and identify the number of songs without user history

In [3]:
from operator import add

In [4]:
song_rdd = sc.textFile(r"kaggle_songs.txt").map(lambda line: line.split(" "))
song_map = song_rdd.map(lambda x:(x[0],x[1]))
count = song_rdd.map(lambda x:1).reduce(add)
count
#print(song_map.take(3))

386213

In [5]:
triplet_map = triplet_rdd.map(lambda x:(x[1],(x[0],x[2])))
join0 = song_map.leftOuterJoin(triplet_map)
print(join0.take(2))
final = join0.map(lambda x:(x[1]))

print(final.take(3))
def liked(col):
    if(col[1] != None):
        return True
    return False
replace = final.filter(liked).map(lambda x:(x[1][0],x[0],x[1][1]))

#replace.collect()
print(replace.take(3))

[('SOAAALJ12AB01828B4', ('12', None)), ('SOAAAQN12AB01856D3', ('16', ('32cf63cf65787ce7e72fc7fda6ee585979af6582', '1')))]
[('12', None), ('16', ('32cf63cf65787ce7e72fc7fda6ee585979af6582', '1')), ('16', ('d1b88940eabd4fab860edf68f0f0842e4f902c78', '1'))]
[('32cf63cf65787ce7e72fc7fda6ee585979af6582', '16', '1'), ('d1b88940eabd4fab860edf68f0f0842e4f902c78', '16', '1'), ('140be58d1ea6d645ed7b317f61197b97975ac1ba', '29', '1')]


In [6]:
nore = replace.map(lambda x:(x[1],x))
noredun = nore.reduceByKey(lambda x,y:x)
count1=noredun.map(lambda x:1).reduce(add)
outcome = count-count1
print("The number of songs without user history is ", outcome )

The number of songs without user history is  223007


## Step 2:
Generate song ratings based on the play_count. For example, if (song_1, 5; song_2, 10; song_3, 5) i.e., song_1 is played 5 times, song_2 is played 10 times and song_3 is played 5 times, the normalized rating score should be 0.25, 0.5 and 0.25 respectively. 
Similarly, generate the rating for all the songs. You may notice that based on all songs, the rating is almost always very low. So, think of the best way to convert song count to ratings. (Hint: Try generating ratings based on each user's song play history)

In [7]:
#user_rdd = sc.textFile(r"kaggle_users.txt") \
 #   .map(lambda line: line.split("\n")) 
#usermap = user_rdd.map(lambda x: x[0])
# usermap.take(5)
usertimes = triplet_rdd.map(lambda x:(x[0],int(x[2]))).reduceByKey(add)
print(usertimes.take(5))
usertimes_map =usertimes.map(lambda x:(x[0],x[1]))
# times = triplet_rdd.map(lambda x:(x[1],1)).reduceByKey(add)
# print(times.take(5))
# time = triplet_rdd.map(lambda x:(x[1],int(x[2]))).reduceByKey(add)
# print(time.take(5))

[('d7083f5e1d50c264277d624340edaaf3dc16095b', 17), ('d68dc6fc25248234590d7668a11e3335534ae4b4', 6), ('fdf6afb5daefb42774617cf223475c6013969724', 10), ('10cbcd627472477dfbec90fb75017f8df6ce84ec', 9), ('30e4a688e6fc9c8bfe55998af3996a909ae34449', 8)]


In [8]:
replace_map = replace.map(lambda x:(x[0],(x[1],x[2])))
join1 = usertimes_map.leftOuterJoin(replace_map)
# print(join1.take(3))
ratings = join1.map(lambda x:(x[0],x[1][1][0],int(x[1][1][1])/x[1][0]))
print(ratings.take(5))
# join1 = times.leftOuterJoin(time)
# # print(join1.take(3))
# ratings = join1.map(lambda x:(x[0],x[1][1]/x[1][0]))
# # print(ratings.take(3))
# ratings.collect()
#ratings.collect()

[('6530c4fc41b9110de5d39fe0355fa103c66385f0', '36681', 0.11363636363636363), ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '75497', 0.18181818181818182), ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '245936', 0.06818181818181818), ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '383719', 0.022727272727272728), ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '129546', 0.1590909090909091)]


In [9]:
import math
final_rating = ratings.map(lambda x:(x[0],x[1],10+math.log(x[2])))
#final_rating.collect()
print(final_rating.take(5))

[('6530c4fc41b9110de5d39fe0355fa103c66385f0', '36681', 7.825248278515839), ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '75497', 8.295251907761575), ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '245936', 7.3144226547498485), ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '383719', 6.2158103660817385), ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '129546', 8.161720515137052)]


## Step 3: 
For a given user_id, rating, recommend 5 other songs from the list. One way to do this is based on another user who liked the same song liked by this user with rating more than the given rating and recommend the 5 songs based on the matched user's rating. 

In [ ]:
# triplet_map1 = triplet_rdd.map(lambda x:(x[1],(x[0],x[2])))
# ratings_map = ratings.map(lambda x:(x[0],x[1]))
# join2 = triplet_map1.leftOuterJoin(ratings_map)
# join2.take(5)
# def liked(col):
#     if(col[1] != None and col[0] != None):
#         return True
#     return False
# whole = join2.filter(liked).map(lambda x:(x[1][0][0],x[0],x[1][1],x[1][0][1]))

# whole.take(5)

# #hi :)

In [10]:
## Take inputs from final ratings from step 2
givenSongId = '36681'; # which song
givenUserId = '6530c4fc41b9110de5d39fe0355fa103c66385f0'; # which User
givenRating = 7.825248278515839; # what is the threshold for "liking"

In [11]:
##    userId, songId and rating
final_collect = final_rating.map(lambda line: (line[0], [line[1], line[2]]))
#################################k###################
#final_collect.collect()
print(final_collect.take(5))

[('6530c4fc41b9110de5d39fe0355fa103c66385f0', ['36681', 7.825248278515839]), ('6530c4fc41b9110de5d39fe0355fa103c66385f0', ['75497', 8.295251907761575]), ('6530c4fc41b9110de5d39fe0355fa103c66385f0', ['245936', 7.3144226547498485]), ('6530c4fc41b9110de5d39fe0355fa103c66385f0', ['383719', 6.2158103660817385]), ('6530c4fc41b9110de5d39fe0355fa103c66385f0', ['129546', 8.161720515137052])]


In [12]:
## Function to filter users who also liked the given song
def liked(col):
    if((col[0] != '') and (col[1][0] == givenSongId) and (float(col[1][1]) > givenRating)):
        return True
    return False
####################################################
## Get all users who liked the given song and what rating they gave it
## above the like threshold
users = final_collect.filter(liked) \
                    .map(lambda line: (line[0], line[1][1])) 
users.collect()

[('6385fa76b90e95f49909701b5effcdc17ea3bc0a', 8.208240530771945),
 ('af71c172969f8918b58b7d160177e5e332332bc5', 8.3905620875659),
 ('80b0aeeb14f49454ad61f5d3b54460aacb0e6cbe', 8.208240530771945),
 ('b43f79fc00f1ff69eb272aa40721c3a1ec7ba911', 7.985096979457735),
 ('70349986733e95f7ca4ce65daa575d673fd26109', 8.054089850944687),
 ('e7a3c5907d77e3f03c2adc9ea168404d8a891a04', 8.341771923396468),
 ('eacd33bf44d39f537591fb06aad0b28dd48f980c', 8.32602356642833),
 ('7c948aa17a6528c319eed401f3559d862032910c', 7.871768294150732),
 ('fbcc48dfceb317106323ad9b959b0e9d98be52ff', 9.106182123977904),
 ('194396fa0779016224bca8503b6ac5eb4617cc2c', 8.495922603223725),
 ('901e60fdedebb03b0b7714719d48f560eb2367e6', 7.920558458320164),
 ('7c71b8809696fc199305bc71b49b0d2ff0b56d54', 8.3905620875659),
 ('42f4f6585483ef3a7fab1314a8b1808236033615', 8.208240530771945),
 ('e512f0042cc25d560e23d3760f024271515a89d4', 8.424463639241582),
 ('183e5fa03bf810c12e9968be67c7657440ab4235', 8.083077387817939),
 ('b52ffe126320

In [13]:
## Get songs that users who liked the given song, listened to
## (songId, [userId, rating])
usersSong = final_collect.join(users)
usersSongs = usersSong.map(lambda line: (line[1][0][0], [line[0], line[1][0][1]]))
# ####################################################

In [14]:
usersSongs.take(5)

[('36681', ['6385fa76b90e95f49909701b5effcdc17ea3bc0a', 8.208240530771945]),
 ('288097', ['6385fa76b90e95f49909701b5effcdc17ea3bc0a', 8.208240530771945]),
 ('385781', ['6385fa76b90e95f49909701b5effcdc17ea3bc0a', 8.208240530771945]),
 ('187518', ['6385fa76b90e95f49909701b5effcdc17ea3bc0a', 8.208240530771945]),
 ('337171', ['6385fa76b90e95f49909701b5effcdc17ea3bc0a', 8.90138771133189])]

In [15]:
## Function to filter not to keep the given song
def notTheGivenSong(row):
    if(row[0] != givenSongId):
        return True
    return False
####################################################
## Next step: Get only those songs that are liked by the users who liked given song
## and their aggregate rating
usersSongsWithoutGiven = usersSongs.filter(notTheGivenSong)
####################################################
usersSongsWithoutGiven.collect()

[('288097', ['6385fa76b90e95f49909701b5effcdc17ea3bc0a', 8.208240530771945]),
 ('385781', ['6385fa76b90e95f49909701b5effcdc17ea3bc0a', 8.208240530771945]),
 ('187518', ['6385fa76b90e95f49909701b5effcdc17ea3bc0a', 8.208240530771945]),
 ('337171', ['6385fa76b90e95f49909701b5effcdc17ea3bc0a', 8.90138771133189]),
 ('340724', ['af71c172969f8918b58b7d160177e5e332332bc5', 8.3905620875659]),
 ('172659', ['af71c172969f8918b58b7d160177e5e332332bc5', 8.3905620875659]),
 ('281084', ['af71c172969f8918b58b7d160177e5e332332bc5', 8.3905620875659]),
 ('35213', ['af71c172969f8918b58b7d160177e5e332332bc5', 8.3905620875659]),
 ('208185', ['80b0aeeb14f49454ad61f5d3b54460aacb0e6cbe', 7.109628242103835]),
 ('208210', ['80b0aeeb14f49454ad61f5d3b54460aacb0e6cbe', 7.109628242103835]),
 ('245050', ['80b0aeeb14f49454ad61f5d3b54460aacb0e6cbe', 8.495922603223725]),
 ('287839', ['80b0aeeb14f49454ad61f5d3b54460aacb0e6cbe', 7.109628242103835]),
 ('303151', ['80b0aeeb14f49454ad61f5d3b54460aacb0e6cbe', 7.109628242103835

In [16]:
#Drop the users from RDD
noSongsInUsers = usersSongsWithoutGiven.map (lambda x: (x[0], float (x[1][1])))
noSongsInUsers.collect()

[('288097', 8.208240530771945),
 ('385781', 8.208240530771945),
 ('187518', 8.208240530771945),
 ('337171', 8.90138771133189),
 ('340724', 8.3905620875659),
 ('172659', 8.3905620875659),
 ('281084', 8.3905620875659),
 ('35213', 8.3905620875659),
 ('208185', 7.109628242103835),
 ('208210', 7.109628242103835),
 ('245050', 8.495922603223725),
 ('287839', 7.109628242103835),
 ('303151', 7.109628242103835),
 ('352646', 7.109628242103835),
 ('377773', 7.109628242103835),
 ('144962', 7.80277542266378),
 ('199054', 7.109628242103835),
 ('206245', 7.109628242103835),
 ('302128', 7.109628242103835),
 ('272257', 6.955477562276577),
 ('373359', 6.955477562276577),
 ('265906', 8.564915474710677),
 ('366243', 6.955477562276577),
 ('50702', 6.955477562276577),
 ('71156', 6.955477562276577),
 ('153916', 6.955477562276577),
 ('39077', 6.955477562276577),
 ('48941', 8.054089850944687),
 ('168176', 7.648624742836523),
 ('60860', 7.920558458320164),
 ('64432', 7.227411277760218),
 ('15718', 7.227411277760

In [17]:
#Getting the song indexes with top 5 aggregated ratings
aggSongs = noSongsInUsers.reduceByKey(add)
aggSongs.takeOrdered(5, key= lambda x: -x[1])

[('328524', 23.75002475774052),
 ('215861', 15.030186700423998),
 ('286747', 14.96304739758637),
 ('36622', 14.36521039683075),
 ('242113', 13.750990474860476)]

In [18]:
##Reordering the songs by switching index and song name
song_names_reorderedd = song_rdd.map(lambda x: (x[1], x[0]))
song_names_reorderedd.take(4)

[('1', 'SOAAADD12AB018A9DD'),
 ('2', 'SOAAADE12A6D4F80CC'),
 ('3', 'SOAAADF12A8C13DF62'),
 ('4', 'SOAAADZ12A8C1334FB')]

In [19]:
#Do a join of the aggregated songs so now we have song index, aggregated ratings, and song name
song_names = aggSongs.join(song_names_reorderedd)
song_names.take(5)

[('281084', (8.3905620875659, 'SOSJRXV12A8C136E1B')),
 ('123485', (7.227411277760218, 'SOHTEDD12A6D4F8215')),
 ('242113', (13.750990474860476, 'SOPQJAZ12A6310F168')),
 ('225526', (8.054089850944687, 'SOOMNQN12AB0185E62')),
 ('71156', (6.955477562276577, 'SOELASR12A6D4FAB10'))]

In [20]:
#Reordering song names so now we have song name, aggregated ratings
song_names_reformatted = song_names.map(lambda x: (x[1][1],x[1][0]))
song_names_reformatted.take(5)

[('SOSJRXV12A8C136E1B', 8.3905620875659),
 ('SOHTEDD12A6D4F8215', 7.227411277760218),
 ('SOPQJAZ12A6310F168', 13.750990474860476),
 ('SOOMNQN12AB0185E62', 8.054089850944687),
 ('SOELASR12A6D4FAB10', 6.955477562276577)]

In [21]:
#Take top 5 from song names list
song_names_reformatted.takeOrdered(5, key= lambda x: -x[1])

[('SOVTLQW12AB0186641', 23.75002475774052),
 ('SONVPTP12A6D4F7A34', 15.030186700423998),
 ('SOSUADS12A58A80A47', 14.96304739758637),
 ('SOCGXXL12B0B808865', 14.36521039683075),
 ('SOPQJAZ12A6310F168', 13.750990474860476)]

## Step 4: 
1. Compute cosine similarity between all pairs of users. 
2. Sort the similarity score and print the top-5 similar users. 
3. If the top-5 user set has an user appearing more than once, ignore that pair and take the next best pair from the sorted list. 
4. For a given user_id, identify the top-5 similar users and hence song recommendations from other user's list. 

In [22]:
ratingrdd = ratings.map(lambda x: (x[0],(x[1],x[2])))
#ratingrdd.take(5)

rddlist = ratingrdd.groupByKey().map(lambda x:(x[0],list(x[1])))

rddlist_key = rddlist.map(lambda x:(1,x))

rddlist_key.take(5)

[(1,
  ('6530c4fc41b9110de5d39fe0355fa103c66385f0',
   [('36681', 0.11363636363636363),
    ('75497', 0.18181818181818182),
    ('245936', 0.06818181818181818),
    ('383719', 0.022727272727272728),
    ('129546', 0.1590909090909091),
    ('253868', 0.022727272727272728),
    ('280679', 0.045454545454545456),
    ('294531', 0.022727272727272728),
    ('357202', 0.022727272727272728),
    ('346138', 0.045454545454545456),
    ('165401', 0.045454545454545456),
    ('176139', 0.022727272727272728),
    ('275127', 0.045454545454545456),
    ('242151', 0.045454545454545456),
    ('327050', 0.022727272727272728),
    ('334240', 0.11363636363636363)])),
 (1,
  ('5a68f7886f7e778490c6f13807039ff4152bcd62',
   [('340434', 0.18181818181818182),
    ('299176', 0.09090909090909091),
    ('93787', 0.09090909090909091),
    ('318282', 0.36363636363636365),
    ('180600', 0.09090909090909091),
    ('286878', 0.18181818181818182)])),
 (1,
  ('6493c305190b52657d4ea3f4adf367ffcf3427af',
   [('37903', 0.0

In [23]:

def pick(col):
    if(col[1][0] == '6530c4fc41b9110de5d39fe0355fa103c66385f0'):
        return True
    return False
rddlist_taruser = rddlist_key.filter(pick)
rddlist_taruser


PythonRDD[67] at RDD at PythonRDD.scala:53

In [24]:
join = rddlist_key.join(rddlist_taruser)
def notTheGivenUser(row):
    if(row[1][0][0] != '6530c4fc41b9110de5d39fe0355fa103c66385f0'):
        return True
    return False
join = join.filter(notTheGivenUser).map(lambda x: x)
join.take(5)

[(1,
  (('5a68f7886f7e778490c6f13807039ff4152bcd62',
    [('340434', 0.18181818181818182),
     ('299176', 0.09090909090909091),
     ('93787', 0.09090909090909091),
     ('318282', 0.36363636363636365),
     ('180600', 0.09090909090909091),
     ('286878', 0.18181818181818182)]),
   ('6530c4fc41b9110de5d39fe0355fa103c66385f0',
    [('36681', 0.11363636363636363),
     ('75497', 0.18181818181818182),
     ('245936', 0.06818181818181818),
     ('383719', 0.022727272727272728),
     ('129546', 0.1590909090909091),
     ('253868', 0.022727272727272728),
     ('280679', 0.045454545454545456),
     ('294531', 0.022727272727272728),
     ('357202', 0.022727272727272728),
     ('346138', 0.045454545454545456),
     ('165401', 0.045454545454545456),
     ('176139', 0.022727272727272728),
     ('275127', 0.045454545454545456),
     ('242151', 0.045454545454545456),
     ('327050', 0.022727272727272728),
     ('334240', 0.11363636363636363)]))),
 (1,
  (('6493c305190b52657d4ea3f4adf367ffcf3427af

In [25]:
def merge(row):
    list1 = row[1][1][1]
    list2 = row[1][0][1]
    
    dict1={}
    for i in list1:
        dict1[i[0]] = []
        dict1[i[0]].append(i[1])
        dict1[i[0]].append(0)
        
    for i in list2:
        if i[0] not in dict1:
            dict1[i[0]] = []
            dict1[i[0]].append(0)
            dict1[i[0]].append(i[1])
        else:
            dict1[i[0]][1] = i[1]
            
    return (row[1][0][0],dict1)
merged = join.map(lambda x : merge(x))
merged.take(5)



[('5a68f7886f7e778490c6f13807039ff4152bcd62',
  {'36681': [0.11363636363636363, 0],
   '75497': [0.18181818181818182, 0],
   '245936': [0.06818181818181818, 0],
   '383719': [0.022727272727272728, 0],
   '129546': [0.1590909090909091, 0],
   '253868': [0.022727272727272728, 0],
   '280679': [0.045454545454545456, 0],
   '294531': [0.022727272727272728, 0],
   '357202': [0.022727272727272728, 0],
   '346138': [0.045454545454545456, 0],
   '165401': [0.045454545454545456, 0],
   '176139': [0.022727272727272728, 0],
   '275127': [0.045454545454545456, 0],
   '242151': [0.045454545454545456, 0],
   '327050': [0.022727272727272728, 0],
   '334240': [0.11363636363636363, 0],
   '340434': [0, 0.18181818181818182],
   '299176': [0, 0.09090909090909091],
   '93787': [0, 0.09090909090909091],
   '318282': [0, 0.36363636363636365],
   '180600': [0, 0.09090909090909091],
   '286878': [0, 0.18181818181818182]}),
 ('6493c305190b52657d4ea3f4adf367ffcf3427af',
  {'36681': [0.11363636363636363, 0],
   

In [26]:
def nozero(row):
    dict_rate = row[1]
    for value in dict_rate.values():
        if(value[0] != 0 and value[1] !=0):
            return True
        return False
    
nozero1 = merged.filter(nozero)
nozero1.take(5)

[('d68df3e077f0f7186a253b9d7fbc46bbab9729c7',
  {'36681': [0.11363636363636363, 0.03571428571428571],
   '75497': [0.18181818181818182, 0],
   '245936': [0.06818181818181818, 0],
   '383719': [0.022727272727272728, 0],
   '129546': [0.1590909090909091, 0],
   '253868': [0.022727272727272728, 0],
   '280679': [0.045454545454545456, 0],
   '294531': [0.022727272727272728, 0],
   '357202': [0.022727272727272728, 0],
   '346138': [0.045454545454545456, 0],
   '165401': [0.045454545454545456, 0],
   '176139': [0.022727272727272728, 0],
   '275127': [0.045454545454545456, 0],
   '242151': [0.045454545454545456, 0.10714285714285714],
   '327050': [0.022727272727272728, 0],
   '334240': [0.11363636363636363, 0],
   '31927': [0, 0.03571428571428571],
   '80562': [0, 0.03571428571428571],
   '121836': [0, 0.03571428571428571],
   '278861': [0, 0.03571428571428571],
   '282293': [0, 0.14285714285714285],
   '47391': [0, 0.03571428571428571],
   '75701': [0, 0.03571428571428571],
   '221689': [0, 

In [27]:
def rate(row):
    list1 = []
    list2 = []
    
    for i in row[1].values():
        list1.append(i[0])
        list2.append(i[1])
        
    return (row[0], list1, list2)

rate1 = nozero1.map(lambda row : rate(row))
rate1.take(5)

[('d68df3e077f0f7186a253b9d7fbc46bbab9729c7',
  [0.11363636363636363,
   0.18181818181818182,
   0.06818181818181818,
   0.022727272727272728,
   0.1590909090909091,
   0.022727272727272728,
   0.045454545454545456,
   0.022727272727272728,
   0.022727272727272728,
   0.045454545454545456,
   0.045454545454545456,
   0.022727272727272728,
   0.045454545454545456,
   0.045454545454545456,
   0.022727272727272728,
   0.11363636363636363,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  [0.03571428571428571,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0.10714285714285714,
   0,
   0,
   0.03571428571428571,
   0.03571428571428571,
   0.03571428571428571,
   0.03571428571428571,
   0.14285714285714285,
   0.03571428571428571,
   0.03571428571428571,
   0.03571428571428571,
   0.03571428571428571,
   0.03571428571428571,
   0.03571428571428571,
   0.07142857142857142,
   0.03571428571428571,
 

In [28]:
from sklearn.metrics.pairwise import cosine_similarity

In [29]:
#Compute cosine similarity between all pairs of users.
cosinesi = rate1.map(lambda x: (x[0], cosine_similarity([x[1],x[2]])[0][1]))
cosinesi.take(5)

[('d68df3e077f0f7186a253b9d7fbc46bbab9729c7', 0.1105541596785133),
 ('dbdf054455bd963891e3e500bd274be913b8bd4e', 0.010189916417594135),
 ('52407d063b39d0fbcf906196ec04d3f488ce6bf4', 0.06185580158543379),
 ('f0f9d219ef4ed210976804133ebeb11a75be9fd6', 0.07945521577046602),
 ('98e1fa0fd12b9ccb837e8b7ed36435a2b7e2b60b', 0.024697456915243728)]

In [30]:
#Sort the similarity score and print the top-5 similar users.
sortby = cosinesi.takeOrdered(5, key= lambda x: -x[1])
sortby

[('38629c8bcf7a3c4b7dc9e18678dbe9b636b92d7a', 0.3914397598210878),
 ('8ca5289fba197c5a2b7e0995e916c30f43fef976', 0.2927393390647722),
 ('fcb364bb1e10c6ef3fa01efdb07b53019ec06c89', 0.27524094128159016),
 ('a4c3b833a60582df9aa5370df1dfc2dd792e4106', 0.2577867603707275),
 ('b56cf3b5e789ac8dfb3165516eabbeb6f3b05022', 0.2544603402301914)]

In [31]:
newmap = final_rating.map(lambda x: (x[0],(x[1],x[2])))
sort = sc.parallelize(sortby)
temsong = sort.join(newmap)
#temsong.take(5)
tempsong = temsong.map(lambda x: (x[0], x[1][1]))
#tempsong.take(5)
finalsong = tempsong.map(lambda x: x[1])
finalsong.take(5)

[('15208', 7.80277542266378),
 ('36681', 8.90138771133189),
 ('98293', 7.80277542266378),
 ('114110', 7.80277542266378),
 ('339441', 7.80277542266378)]

In [33]:
def newfilter(col):
    if(col[0] == givenUserId):
        return True
    return False
userfilter = final_rating.filter(newfilter).map(lambda x:(x[1],x[2]))
userfilter.collect()

[('36681', 7.825248278515839),
 ('75497', 8.295251907761575),
 ('245936', 7.3144226547498485),
 ('383719', 6.2158103660817385),
 ('129546', 8.161720515137052),
 ('253868', 6.2158103660817385),
 ('280679', 6.908957546641684),
 ('294531', 6.2158103660817385),
 ('357202', 6.2158103660817385),
 ('346138', 6.908957546641684),
 ('165401', 6.908957546641684),
 ('176139', 6.2158103660817385),
 ('275127', 6.908957546641684),
 ('242151', 6.908957546641684),
 ('327050', 6.2158103660817385),
 ('334240', 7.825248278515839)]

In [37]:
filtersong = finalsong.leftOuterJoin(userfilter)
def delete(col):
    if(col[1][1] == None):
        return True
    return False
filteredsong = filtersong.filter(delete).map(lambda x:(x[0],x[1][0]))
filteredsong.collect()

[('98293', 7.80277542266378),
 ('41965', 7.05556102083356),
 ('114110', 7.80277542266378),
 ('55968', 6.362413840273614),
 ('200841', 7.05556102083356),
 ('43841', 5.212508257217954),
 ('217954', 7.80277542266378),
 ('336560', 7.487694376023885),
 ('10264', 8.470604795239437),
 ('72158', 8.431384082086154),
 ('328524', 7.775376448475667),
 ('111532', 6.362413840273614),
 ('90943', 8.009638088373853),
 ('353980', 6.362413840273614),
 ('203750', 7.0822292679157215),
 ('92684', 6.362413840273614),
 ('231080', 6.362413840273614),
 ('195346', 5.212508257217954),
 ('22749', 7.775376448475667),
 ('100194', 6.362413840273614),
 ('310521', 5.9056554377779),
 ('119952', 7.487694376023885),
 ('238513', 6.362413840273614),
 ('36622', 6.311120545886064),
 ('334526', 5.212508257217954),
 ('9987', 4.918595635015537),
 ('311987', 6.362413840273614),
 ('222666', 7.05556102083356),
 ('81001', 5.611742815575482),
 ('67533', 7.05556102083356),
 ('310842', 7.751808979071753),
 ('282859', 4.918595635015537)

In [38]:
#For a given user_id, identify the top-5 similar users and hence song recommendations from other user's list.
songrate = filteredsong.groupByKey().map(lambda x: (x[0], list(x[1])))
songrate1= songrate.map(lambda x: (x[0], (sum(x[1])/len(x[1]))))
#songrate1.take(5)
Top5Song = songrate1.takeOrdered(5, key= lambda x: -x[1])
Top5Song.collect()

[('10264', 8.470604795239437),
 ('72158', 8.431384082086154),
 ('165865', 8.308323989328928),
 ('90943', 8.009638088373853),
 ('121733', 7.863034614181977)]